In [1]:
import numpy as np
from keras import layers
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import ZeroPadding2D
from keras.layers import AveragePooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import BatchNormalization
from keras.models import Model
import keras.preprocessing.image
import keras.backend as K
#from keras.applications.Detect import ResNet50, preprocess_input
#from keras.applications.xception import Xception, preprocess_input
#from keras.applications.vgg19 import VGG19, preprocess_input
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam

import os
import sys
import glob
import tensorflow as tf
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
import keras.backend.tensorflow_backend as KTF
from keras import optimizers 
import datetime
import re
import math
import pandas as pd
import json
from pixel_shuffler import PixelShuffler

/home/zhanggw/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
Defect_class = [
    'defect_1',
    'defect_2',
    'defect_3',
    'defect_4',
    'defect_5',
    'defect_6',
    'defect_7',
    'defect_8',
    'defect_9',
    'defect_10',
    'norm',
]
IMAGE_SHAPE = (640,640,3)
ENCODER_DIM = 3200
BATCH_SIZE = 16
CLASS_COUNT = 11
BASE_DIR = './logs'

ENCODER_WEIGHT = './logs/encoder_20180828T1034.h5'
CLASSFY_WEIGHT = './logs/classfy_20180828T1034.h5'


EPOCH_INIT = 0
EPOCH_TOTAL = 4000
EPOCH_STEP = 1000


In [3]:
def preprocess_input(x):
    x = x/255.0
    return x

In [4]:
import imgaug as ia
from imgaug import augmenters as iaa
import logging
import skimage.io
import skimage.color

class ImgGenerator(object):
    def __init__(self, samples, class_mapping, batch_size, augment=True, shuffle=True):
        self.batch_size = batch_size
        self.augment = augment
        self.shuffle = shuffle
        self.samples = samples
        self.class_mapping = class_mapping
        self.classcnt = len(self.class_mapping.keys())
        print("Class count:{}".format(self.classcnt))
        self.encoder = encoder
        self.encoder.load_weights(ENCODER_WEIGHT)
        if self.augment:
            self.augment_prepare()

    def augment_prepare(self):
        ia.seed(20180827)
        self.ia_seq = iaa.Sequential(
            [
                iaa.Affine(rotate=(-10, 10), shear=(-5, 5)),
            ],
            random_order=True,
        )
    def augment_image(self, batch_images):
        seq_det = self.ia_seq.to_deterministic()
        images_aug = seq_det.augment_images(batch_images)
        return images_aug

    def len(self):
        return len(self.samples)

    def load_image(self, image_id):
        filepath = self.samples[image_id]
        classid = self.class_mapping[filepath.split('/')[-2]]
        img = skimage.io.imread(filepath)
        if img.ndim != 3:
            img = skimage.color.gray2rgb(img)
        img = keras.preprocessing.image.img_to_array(img)
        #print("Imgpath:{}\n label:{}".format(filepath, classid))
        return img, classid

    def flow(self):
        b = 0
        image_index = -1
        image_ids = np.arange(len(self.samples))

        while True:
            try:
                image_index = (image_index + 1) % len(image_ids)
                if self.shuffle and image_index == 0:
                    np.random.shuffle(image_ids)

                image_id = image_ids[image_index]
                img, classid = self.load_image(image_id)

                if self.augment:
                    img = self.augment_image(batch_images)
                img = preprocess_input(img)
                if b == 0:
                    #batch_images = np.zeros((self.batch_size,) + img.shape, dtype=np.float32)
                    batch_images = np.zeros((self.batch_size,) + (20, 20, 128*12), dtype=np.float32)
                    batch_labels = np.zeros((self.batch_size, self.classcnt), dtype=np.int32)
               
                block_images = np.zeros((12,) + (640, 640, 3), dtype=np.float32)
                for i in range(3):
                    for j in range(4):
                        block_images[i*4+j] = img[i*640:(i+1)*640, j*640:(j+1)*640, :]
                preds = self.encoder.predict_on_batch(block_images)
                preds = np.reshape(preds, (20, 20, 128*12))

                batch_images[b] = preds
                batch_labels[b, classid] = 1

                b += 1
                if b >= self.batch_size:
                    inputs = batch_images
                    outputs = batch_images   #batch_labels
                    yield inputs, outputs
                    b = 0
            except:
                logging.exception("Error for image {}".format(image_index))
                raise

In [5]:
class_mapping = {defect:classid for classid, defect in enumerate(Defect_class)}
print(class_mapping)

{'defect_1': 0, 'defect_2': 1, 'defect_3': 2, 'defect_4': 3, 'defect_5': 4, 'defect_6': 5, 'defect_7': 6, 'defect_8': 7, 'defect_9': 8, 'defect_10': 9, 'norm': 10}


In [6]:
samples_data = json.load(open('samples.json'))
train_files = samples_data['train']
valid_files = samples_data['valid']
#print(train_files)

In [7]:
def conv(filters):
    def block(x):
        x = Conv2D(filters, kernel_size=5, strides=2, padding='same')(x)
        x = LeakyReLU(0.1)(x)
        return x
    return block

def upscale(filters):
    def block(x):
        x = Conv2D(filters*4, kernel_size=3, padding='same')(x)
        x = LeakyReLU(0.1)(x)
        x = PixelShuffler()(x)
        return x
    return block

def Encoder():
    input_ = Input(shape=IMAGE_SHAPE)
    x = input_

    x = conv(128)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = conv(256)(x)

    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = conv(512)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = conv(1024)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Flatten()(x)
    x = Dense(ENCODER_DIM)(x)
    #x = Dense(4*4*256)(x)
    x = Reshape((10,10,32))(x)
    x = upscale(128)(x)
    return Model(input_, x)

def Decoder():
    input_ = Input(shape=(20,20,128))
    x = input_
    x = upscale(256)(x)
    x = upscale(128)(x)
    x = upscale(128)(x)
    x = upscale(64)(x)
    x = upscale(64)(x)
    x = Conv2D(3, kernel_size=5, padding='same', activation='sigmoid')(x)
    return Model(input_, x)

def Classfy():
    input_ = Input(shape=(20, 20, 128*12))
    x = input_
    x = Conv2D(1024, (3, 3), strides=(2, 2), name='gw_conv01')(x)
    x = Activation('relu')(x)
    x = BatchNormalization(axis=3)(x)

    x = Conv2D(512, (3, 3), strides=(2, 2), name='gw_conv02')(x)
    x = Activation('relu')(x)
    x = BatchNormalization(axis=3)(x)

    x = Conv2D(256, (3, 3), strides=(2, 2), name='gw_conv03')(x)
    x = Activation('relu')(x)
    x = BatchNormalization(axis=3)(x)
    x = Flatten()(x)
 
    x = Dense(CLASS_COUNT, activation='softmax', name='predict')(x)
    model = Model(inputs=input_, outputs=x)
    return model

def detect_model():
    return classfy

In [8]:
encoder = Encoder()
decoder = Decoder()
classfy = Classfy()
#print(encoder.summary())
#print(decoder.summary())
#print(classfy.summary())

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [9]:
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def fbeta_score(y_true, y_pred, beta=1):
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score

def FScore2(y_true, y_pred):
    return fbeta_score(y_true, y_pred, beta=2)

In [10]:
class WeightSaver(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        dt ='{:%Y%m%dT%H%M}'.format(datetime.datetime.now())
        #encoder.save_weights("./logs/encoder_{:%.4d}_{}.h5".format(epoch, dt))
        classfy.save_weights("./logs/classfy_{}.h5".format(dt))

In [11]:
train_generator = ImgGenerator(train_files, class_mapping, BATCH_SIZE)
valid_generator = ImgGenerator(valid_files, class_mapping, BATCH_SIZE, augment=False, shuffle=False)

Class count:11


OSError: Unable to open file (unable to open file: name = 'logs/encoder.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
detectmodel = detect_model()
detectmodel.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy', precision, FScore2])

In [ ]:
log_dir = os.path.join(BASE_DIR, os.path.splitext(os.path.basename(__file__))[0])
checkout_path = os.path.join(log_dir, "{}_*epoch*.h5".format('classfy'))
checkout_path = checkout_path.replace("*epoch*", "{epoch:04d}-{val_acc:.2f}")

In [ ]:
modelfiles = glob.glob(os.path.join(log_dir, "./{}_*.h5".format('classfy'))
modelfiles = sorted(modelfiles)
if len(modelfiles) > 0:
    modelfile = modelfiles[-1]
    print("Load Weight: {}".format(modelfile))
    detectmodel.load_weights(modelfile, by_name=True)


In [ ]:
callbacks = [
    TensorBoard(log_dir=log_dir, write_images=False),
    ModelCheckpoint(checkout_path, monitor='acc', verbose=1, save_weights_only=True, save_best_only=False),
    #EarlyStopping(monitor='acc', patience=50000, verbose=1)
    WeightSaver(),
]


In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = '5'
history_tl = self.model.fit_generator(
                generator = train_generator.flow(),
                initial_epoch = EPOCH_INIT,
                epochs = EPOCH_TOTAL,
                steps_per_epoch = EPOCH_STEP,
                validation_data = valid_generator.flow(),
                validation_steps = valid_generator.len()//BATCH_SIZE,
                verbose = 1,
                callbacks = callbacks)
